# Web Mining and Applied NLP (44-620)

## Final Project: Article Summarizer

### Student Name: David Rodriguez-Mayorquin

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

You should bring in code from previous assignments to help you answer the questions below.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

**Question 1**

 Find on the internet an article or blog post about a topic that interests you and you are able to get the text for using the technologies we have applied in the course.  Get the html for the article and store it in a file (which you must submit with your project)

I used the Spotify API to retrieve saved tracks from my own Spotify profile. The data was returned as JSON, which I processed  and converted into HTML file (saved_tracks.html). The file displays my saved Spotify tracks with clickable links to the Spotify pages for each song.

**Question 2** 

Read in your article's html source from the file you created in question 1 and do sentiment analysis on the article/post's text (use `.get_text()`).  Print the polarity score with an appropriate label.  Additionally print the number of sentences in the original article (with an appropriate label)

In [8]:
from bs4 import BeautifulSoup
from textblob import TextBlob

# Read in the HTML file
with open("saved_tracks.html", "r", encoding="utf-8") as file:
    html_content = file.read()

# Extract text from the HTML file
soup = BeautifulSoup(html_content, "html.parser")
text = soup.get_text()

# Split text into sentences
sentences = text.replace("\n", ". ").split(". ")
print("Extracted Sentences:", sentences)
print("Number of Sentences:", len(sentences))

# Remove empty strings from sentences
sentences = [sentence for sentence in sentences if sentence.strip()]
print("Filtered Sentences:", sentences)
print("Number of Sentences:", len(sentences))

# Perform sentiment analysis
blob = TextBlob(text)
polarity = blob.sentiment.polarity  # Polarity score (-1 to 1)

# Print results
print("Polarity Score:", polarity)
print("Number of Sentences in the Article:", len(sentences))


Extracted Sentences: ['', '', '', '', '', '', 'My Spotify Tracks', '', '', 'My Saved Spotify Tracks', '', '1: Alejandra Guzman - Soy Sólo Un Secreto2: Culture Club - Time (Clock Of The Heart) - Remastered 20033: F.R', "David - Words - Original Version 19834: The Vantages - Leather Jacket5: MGT - Knowing Me Knowing You6: Belinda Carlisle - Circle In The Sand7: Cher - When Love Calls Your Name8: Kim Carnes - I'll Be There Where The Heart Is9: Billie Eilish - CHIHIRO10: Edie Brickell & New Bohemians - What I Am", '', '', '', '']
Number of Sentences: 17
Filtered Sentences: ['My Spotify Tracks', 'My Saved Spotify Tracks', '1: Alejandra Guzman - Soy Sólo Un Secreto2: Culture Club - Time (Clock Of The Heart) - Remastered 20033: F.R', "David - Words - Original Version 19834: The Vantages - Leather Jacket5: MGT - Knowing Me Knowing You6: Belinda Carlisle - Circle In The Sand7: Cher - When Love Calls Your Name8: Kim Carnes - I'll Be There Where The Heart Is9: Billie Eilish - CHIHIRO10: Edie Bric

Polarity score: 0.3372 indicates that the overall sentiment is midly positive, most likely due to neutral song titles
the final number of sentences count reflects only the non-empty sentences after filtering

**Question3**

Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels)

4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels).

5. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

6. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

7. Using the histograms from questions 5 and 6, decide a "cutoff" score for tokens and lemmas such that fewer than half the sentences would have a score greater than the cutoff score.  Record the scores in this Markdown cell

* Cutoff Score (tokens): 
* Cutoff Score (lemmas):

Feel free to change these scores as you generate your summaries.  Ideally, we're shooting for at least 6 sentences for our summary, but don't want more than 10 (these numbers are rough estimates; they depend on the length of your article).

8. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on tokens) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

9. Print the polarity score of your summary you generated with the token scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

10. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on lemmas) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

11. Print the polarity score of your summary you generated with the lemma scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

12.  Compare your polarity scores of your summaries to the polarity scores of the initial article.  Is there a difference?  Why do you think that may or may not be?.  Answer in this Markdown cell.  

13. Based on your reading of the original article, which summary do you think is better (if there's a difference).  Why do you think this might be?